In [ ]:
!pip install pandas requests beautifulsoup4 sqlalchemy


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://www.indeed.com/jobs?q=software+engineer&start="

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

def scrape_page(start):
    url = BASE_URL + str(start)
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    jobs = []

    cards = soup.find_all("div", class_="job_seen_beacon")

    for card in cards:
        title = card.find("h2").text.strip()

        company = card.find("span", class_="companyName")
        company = company.text.strip() if company else None

        location = card.find("div", class_="companyLocation")
        location = location.text.strip() if location else None

        summary = card.find("div", class_="job-snippet")
        summary = summary.text.strip() if summary else None

        jobs.append({
            "title": title,
            "company": company,
            "location": location,
            "summary": summary
        })

    return jobs


all_jobs = []

for start in range(0, 50, 10):
    print("Scraping page:", start)
    all_jobs.extend(scrape_page(start))
    time.sleep(2)

df = pd.DataFrame(all_jobs)
df.to_csv("/content/raw_jobs.csv", index=False)

df.head()


Scraping page: 0
Scraping page: 10
Scraping page: 20
Scraping page: 30
Scraping page: 40


""


In [ ]:
from google.colab import files
files.download("/content/raw_jobs.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install spacy nltk
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!wget https://raw.githubusercontent.com/Mlawrence95/LinkedIn-Tech-Job-Data/main/jobs.csv -O jobs.csv


--2026-01-29 15:23:19--  https://raw.githubusercontent.com/Mlawrence95/LinkedIn-Tech-Job-Data/main/jobs.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76763806 (73M) [text/plain]
Saving to: ‘jobs.csv’

jobs.csv            100%[===================>]  73.21M   228MB/s    in 0.3s    

2026-01-29 15:23:20 (228 MB/s) - ‘jobs.csv’ saved [76763806/76763806]



In [ ]:
import pandas as pd

df = pd.read_csv("jobs.csv")
print(df.shape)
df.head()


(8261, 18)


,Employment type,Industries,Job function,Seniority level,company,company_id,context,date,description,education,location,months_experience,post_id,post_url,sal_high,sal_low,salary,title
0,Full-time,Broadcast Media,Information Technology,Mid-Senior level,CyberCoders,21836.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Job Title: Senior Data Engineer Location: Alex...,bachelor degree,"Alexandria, VA",60.0,2632814552,https://www.linkedin.com/jobs/view/senior-data...,NaN,NaN,NaN,Senior Data Engineer
1,Full-time,"Hospital & Health Care, Medical Devices, and P...",Engineering and Information Technology,Not Applicable,Johnson & Johnson,1207.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,"Ethicon, part of Johnson & Johnson Medical Dev...",bachelor degree,"Santa Clara, CA",96.0,2632810866,https://www.linkedin.com/jobs/view/principal-f...,NaN,NaN,NaN,Principal Full Stack Software Engineer.
2,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Washington, DC",84.0,2632812746,https://www.linkedin.com/jobs/view/senior-soft...,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer
3,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Reston, VA",84.0,2632817427,https://www.linkedin.com/jobs/view/senior-soft...,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer
4,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Irving, TX",84.0,2632817426,https://www.linkedin.com/jobs/view/senior-soft...,175000.0,108000.0,"$108,000.00/yr - $175,000.00/yr",Senior Software Engineer


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("moyukhbiswas/job-postings-dataset")

print("Path to dataset files:", path)

100%|██████████| 12.6k/12.6k [00:00<00:00, 7.68MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/moyukhbiswas/job-postings-dataset/versions/1


In [ ]:
import os

base_path = "/root/.cache/kagglehub/datasets/moyukhbiswas/job-postings-dataset/versions/1"

os.listdir(base_path)


['job_data_merged_1.csv']

In [ ]:
import pandas as pd

df = pd.read_csv("/content/jobs.csv")

print(df.shape)
df.head()



(8261, 18)


,Employment type,Industries,Job function,Seniority level,company,company_id,context,date,description,education,location,months_experience,post_id,post_url,sal_high,sal_low,salary,title
0,Full-time,Broadcast Media,Information Technology,Mid-Senior level,CyberCoders,21836.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Job Title: Senior Data Engineer Location: Alex...,bachelor degree,"Alexandria, VA",60.0,2632814552,https://www.linkedin.com/jobs/view/senior-data...,NaN,NaN,NaN,Senior Data Engineer
1,Full-time,"Hospital & Health Care, Medical Devices, and P...",Engineering and Information Technology,Not Applicable,Johnson & Johnson,1207.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,"Ethicon, part of Johnson & Johnson Medical Dev...",bachelor degree,"Santa Clara, CA",96.0,2632810866,https://www.linkedin.com/jobs/view/principal-f...,NaN,NaN,NaN,Principal Full Stack Software Engineer.
2,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Washington, DC",84.0,2632812746,https://www.linkedin.com/jobs/view/senior-soft...,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer
3,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Reston, VA",84.0,2632817427,https://www.linkedin.com/jobs/view/senior-soft...,189000.0,120000.0,"$120,000.00/yr - $189,000.00/yr",Senior Software Engineer
4,Full-time,"Computer Hardware, Computer Software, and Info...",Engineering and Information Technology,Not Applicable,Microsoft,1035.0,"{""@context"": ""http://schema.org"", ""@type"": ""Jo...",NaN,Microsoft’s WCB health team is looking for a S...,bachelor degree,"Irving, TX",84.0,2632817426,https://www.linkedin.com/jobs/view/senior-soft...,175000.0,108000.0,"$108,000.00/yr - $175,000.00/yr",Senior Software Engineer


In [ ]:
df.isna().sum()


,0
Employment type,493
Industries,1089
Job function,1089
Seniority level,1089
company,0
company_id,481
context,576
date,635
description,481
education,734


In [ ]:
# Keep track of whether salary is present
df["has_salary"] = df["sal_high"].notna()

# Fill education
df["education"] = df["education"].fillna("unknown")

# Experience: fill with median
df["months_experience"] = df["months_experience"].fillna(
    df["months_experience"].median()
)

# Description: empty string for NLP
df["description"] = df["description"].fillna("")


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"[^a-z0-9+.# ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_description"] = df["description"].apply(clean_text)


In [ ]:
df[["months_experience", "education", "clean_description"]].isna().sum()


,0
months_experience,0
education,0
clean_description,0


In [ ]:
SKILLS = [
    "python", "java", "c++", "c#", "javascript", "typescript",
    "sql", "mysql", "postgresql", "mongodb",
    "aws", "azure", "gcp",
    "docker", "kubernetes",
    "react", "angular", "vue", "node", "django", "flask",
    "spark", "hadoop",
    "machine learning", "deep learning",
    "nlp", "data analysis", "data analytics",
    "pandas", "numpy", "scikit learn",
    "tensorflow", "pytorch",
    "power bi", "tableau", "excel"
]


In [ ]:
def extract_skills(text):
    found = []
    for skill in SKILLS:
        if skill in text:
            found.append(skill)
    return found

df["skills"] = df["clean_description"].apply(extract_skills)

df[["clean_description", "skills"]].head()


,clean_description,skills
0,job title senior data engineer location alexan...,"[python, sql, aws, spark, machine learning]"
1,ethicon part of johnson johnson medical device...,"[python, java, c++, javascript, sql, mysql, re..."
2,microsoft s wcb health team is looking for a s...,"[java, c#, javascript, aws, azure, gcp]"
3,microsoft s wcb health team is looking for a s...,"[java, c#, javascript, aws, azure, gcp]"
4,microsoft s wcb health team is looking for a s...,"[java, c#, javascript, aws, azure, gcp]"


In [ ]:
from collections import Counter

all_skills = []
for row in df["skills"]:
    all_skills.extend(row)

Counter(all_skills).most_common(20)


[('python', 4404),
 ('java', 3350),
 ('sql', 2989),
 ('excel', 2844),
 ('aws', 2587),
 ('machine learning', 2523),
 ('c++', 1416),
 ('javascript', 1317),
 ('kubernetes', 1256),
 ('azure', 1187),
 ('docker', 1110),
 ('spark', 1048),
 ('react', 974),
 ('c#', 809),
 ('data analysis', 771),
 ('deep learning', 743),
 ('node', 717),
 ('hadoop', 646),
 ('tensorflow', 629),
 ('gcp', 595)]